In [ ]:
# inicia bibliotecas e local do arquivo
import pandas as pd
import numpy as np
df = pd.read_csv('./vendas_modificado.csv')
df.head(20)

In [ ]:
#verifica os tipos
df.dtypes


In [ ]:
# verifica nulos
df.isnull().sum()

In [ ]:
# verifica os erros de digitação na coluna valor
erros_valor = df[~df['valor'].str.replace('R$', '', regex=False).str.replace(',', '.').str.strip().str.match(r'^\d+(\.\d+)?$')]


print(erros_valor[['valor']].head(10))


In [ ]:
# corige os erros dos valores
df = df[df['valor'].str.replace('R$', '', regex=False).str.replace(',', '.').str.strip().str.match(r'^\d+(\.\d+)?$')]

df['valor'] = df['valor'].str.replace('R$', '', regex=False).str.replace(',', '.').astype(float)



In [ ]:
#padroniza data hora e frete
df['data'] = pd.to_datetime(df['data'], errors='coerce')


df['hora'] = pd.to_datetime(df['hora'], format='%H:%M:%S', errors='coerce').dt.strftime('%H:%M:%S')

df['frete'] = df['frete'].fillna(0.0)

In [ ]:
# verifica nulos
df.isnull().sum()

In [ ]:
# corrige os totais que estão com o valor de Nan com base na formula e verifica se ainda existem nulos 
# como não existem valores nem quantidades nulas não é feita nenhuma verificação
# Fórmula: total = valor * quantidade + frete
#faltando_total = df['total'].isna()
#df.loc[faltando_total, 'total'] = df.loc[faltando_total, 'valor'] * df.loc[faltando_total, 'quantidade'] + df.loc[faltando_total, 'frete']

df.isnull().sum()

In [ ]:
# dropa os nulos restantes 
df = df.dropna()


In [ ]:
# Corrige os valores nulos na coluna 'total' usando a fórmula correta
faltando_total = df['total'].isna()
df.loc[faltando_total, 'total'] = (df.loc[faltando_total, 'valor'] * df.loc[faltando_total, 'quantidade'] + df.loc[faltando_total, 'frete']).round(2)

# Recalcula todos os valores de 'total' e padroniza com duas casas decimais
df['total'] = (df['valor'] * df['quantidade'] + df['frete']).round(2)

# Calcula 'total_calculado' como referência de verificação
df['total_calculado'] = (df['valor'] * df['quantidade'] + df['frete']).round(2)

# Verifica inconsistências entre 'total' e 'total_calculado'
df_totais_errados = df[~np.isclose(df['total'], df['total_calculado'])]

# Exibe registros com discrepâncias
print(df_totais_errados[['valor', 'quantidade', 'frete', 'total', 'total_calculado']])


In [ ]:
# remove resultados maiores que 999 objetivo é de manter todos os resultaods no maximoa té a casa dos cem
df = df[df['total'] <= 999]

In [ ]:
# exibe o data frame
df.head(30)

In [ ]:
# Verificando os status 
df['status'].value_counts()

In [ ]:
#confere se ainda existem nulos
df.isnull().sum()

In [ ]:
# Corrigindo e padronizando os nomes dos status
df['status'] = df['status'].replace({
    'Pgto Confirmado': 'Pagamento Confirmado',
    'aguardando pagamento': 'Aguardando Pagamento',
    'Aguardando Pgto': 'Aguardando Pagamento',
    'AP': 'Aguardando Pagamento',
    'Sep': 'Em Separação',
    'Separando': 'Em Separação',
    'PC': 'Pagamento Confirmado',
    'Entg': 'Entregue',
    'Entregue com Sucesso': 'Entregue',
    'Transp': 'Em Transporte',
    'Transportando': 'Em Transporte'
})


In [ ]:
# Verificando como ficou a distribuição dos status após a padronização
df['status'].value_counts()


In [ ]:
df['pagamento'].value_counts()

In [ ]:
# verificando produtos
print(df['produto'].value_counts().to_string())


In [ ]:
# Dicionário de correções: chaves = nomes errados / valores = nomes corretos
correcoes_produtos = {
    # Corrigindo com símbolos e variações comuns
    'Pasta de Dente#$@!': 'Pasta de Dente',
    'Queijo Mussarela#$@!': 'Queijo Mussarela',
    'Sabonete#$@!': 'Sabonete',
    'Manteiga#$@!': 'Manteiga',
    'Café#$@!': 'Café',
    'Açúcar#$@!': 'Açúcar',
    'Desinfetante#$@!': 'Desinfetante',
    'Papel Toalha#$@!': 'Papel Toalha',
    'Condicionador#$@!': 'Condicionador',
    'Molho de Tomate#$@!': 'Molho de Tomate',
    'Água Mineral#$@!': 'Água Mineral',
    'Refrigerante#$@!': 'Refrigerante',
    'Presunto#$@!': 'Presunto',
    'Cerveja#$@!': 'Cerveja',
    'Biscoito Recheado#$@!': 'Biscoito Recheado',
    'Macarrão#$@!': 'Macarrão',
    'Óleo de Soja#$@!': 'Óleo de Soja',
    'Vinho#$@!': 'Vinho',
    'Sal#$@!': 'Sal',
    'Suco de Laranja#$@!': 'Suco de Laranja',
    'Shampoo#$@!': 'Shampoo',
    'Farinha de Trigo#$@!': 'Farinha de Trigo',
    'Detergente#$@!': 'Detergente',
    'Pão de Forma#$@!': 'Pão de Forma',
    'Arroz#$@!': 'Arroz',
    'Feijão#$@!': 'Feijão',
    'Amaciante#$@!': 'Amaciante',
    'Leite Integral#$@!': 'Leite Integral',
    'Papel Higiênico#$@!': 'Papel Higiênico',
    'Sabão em Pó#$@!': 'Sabão em Pó',
    'Carvão#$@!': 'Carvão',
    'Condicioiador#$@!': 'Condicionador',

    # Corrigindo erros de digitação
    'Refrigkrante': 'Refrigerante',
    'Scl': 'Sal',
    'Papel qoalha': 'Papel Toalha',
    'Desinfetanue': 'Desinfetante',
    'Qbeijo Mussarela': 'Queijo Mussarela',
    'Água Mineras': 'Água Mineral',
    'Presuntd': 'Presunto',
    'Macawrão': 'Macarrão',
    'Sucoyde Laranja': 'Suco de Laranja',
    'Clfé': 'Café',
    'Desinfekante': 'Desinfetante',
    'Majarrão': 'Macarrão',
    'Cnfé': 'Café',
    'Farinha de Tripo': 'Farinha de Trigo',
    'Manteigt': 'Manteiga',
    'zabonete': 'Sabonete',
    'Mqcarrão': 'Macarrão',
    'Arroc': 'Arroz',
    'tal': 'Sal',
    'Sabonepe': 'Sabonete',
    'Papel Twalha': 'Papel Toalha',
    'Água Mineual': 'Água Mineral',
    'Condibionador': 'Condicionador',
    'Caft': 'Café',
    'ieijão': 'Feijão',
    'Açúcaz': 'Açúcar',
    'Amaciayte': 'Amaciante',
    'Caff': 'Café',
    'Mopho de Tomate': 'Molho de Tomate',
    'Molmo de Tomate': 'Molho de Tomate',
    'Biscoitq Recheado': 'Biscoito Recheado',
    'Macirrão': 'Macarrão',
    'Deqergente': 'Detergente',
    'Cafc': 'Café',
    'Queijo Mussarelz': 'Queijo Mussarela',
    'Deterwente': 'Detergente',
}

# Aplicando as correções no DataFrame
df['produto'] = df['produto'].replace(correcoes_produtos)


In [ ]:
# dropa duplicatas
df = df.drop_duplicates()


In [ ]:
# verificando cidades
df['cidade'].value_counts()

In [ ]:
# verificando pais
df['pais'].value_counts()

In [ ]:
# verificando marcas
print(df['marca'].value_counts().to_string())


In [ ]:

# verificando clientes
print(df['cliente'].value_counts().to_string())

In [ ]:
# exibe o data frame
df.head(50)

In [ ]:
# transforma o data frame em csv
df.to_csv('vendas_tratado.csv', index=False)


Relatório de Limpeza e Padronização de Dados 
Durante a análise e tratamento do conjunto de dados, foram identificados e corrigidos diversos problemas que comprometiam a integridade e a qualidade das informações. Abaixo estão as principais etapas realizadas:

1. Leitura e Inspeção Inicial
Foi realizada a importação do arquivo vendas_modificado.csv.

Analisaram-se os tipos de dados e valores nulos.

2. Tratamento da Coluna valor
Identificados erros de digitação e formatação na coluna valor, como presença de símbolos ("R$") e vírgulas como separador decimal.

As entradas inválidas foram removidas.

Os valores foram convertidos corretamente para o tipo float.

3. Padronização de Datas, Horários e Frete
A coluna data foi convertida para o formato datetime.

A coluna hora foi formatada para o padrão HH:MM:SS.

A coluna frete teve valores nulos preenchidos com 0.0.

4. Correção da Coluna total
Valores ausentes em total foram recalculados usando a fórmula:
total = valor * quantidade + frete.

Valores incorretos foram ajustados com base na mesma fórmula.

5. Eliminação de Duplicatas e Valores Faltantes
Registros com valores ainda nulos foram removidos.

Também foram eliminadas duplicatas do DataFrame.

6. Padronização dos Status de Pagamento
A coluna status apresentava inconsistência nos nomes.

Os valores foram padronizados para categorias claras como:
Pagamento Confirmado, Aguardando Pagamento, Em Separação, Em Transporte, Entregue, entre outros.

7. Correção de Nomes de Produtos
Diversos produtos apresentavam caracteres especiais, erros de digitação ou variações no nome.

Foi criado um dicionário de mapeamento para corrigir e padronizar todos esses valores.

8. Análises Finais
Foi realizada uma checagem nas colunas de cidade, país, marca e cliente para garantir consistência nos dados.

Resumo
O processo de limpeza resultou em um DataFrame com:

Tipos de dados consistentes

Nomes padronizados

Cálculos corretos

Ausência de valores nulos e duplicados

Essas correções garantem maior confiabilidade para análises futuras com base nesse conjunto de dados.